# Camera calibration - DLT Algorithm implementation

Use the **Direct Linear Transform** to map 3D-2D correspondences. The camera relates 3D world points and image points by the **Camera Matrix**. This matrix is defined as: 
\begin{equation*}
\begin{bmatrix}
c_{11} &c_{12}  &c_{13}  &c_{34} \\ 
c_{21} &c_{22}  &c_{23}  & c_{23}\\ 
c_{31} & c_{32} & c_{33} & c_{34}
\end{bmatrix}  = 
\begin{bmatrix}
f_x & 0 & u_o\\ 
0 &f_y  &v_o \\ 
0 &0  &1 
\end{bmatrix} \cdot
\begin{bmatrix}
r_{11} &r_{12}  &r_{13}  &t_{34} \\ 
r_{21} &r_{22}  &r_{23}  & t_{23}\\ 
r_{31} &r_{32} &r_{33} & t_{34}
\end{bmatrix} 
\end{equation*}

The camera projection is: 
\begin{equation*}
\begin{bmatrix}
\tilde{u}\\ 
\tilde{v}\\ 
1
\end{bmatrix} = 
\begin{bmatrix}
c_{11} &c_{12}  &c_{13}  &c_{34} \\ 
c_{21} &c_{22}  &c_{23}  & c_{23}\\ 
c_{31} & c_{32} & c_{33} & c_{34}
\end{bmatrix}  \cdot 
\begin{bmatrix}
X_w\\ 
Y_w\\ 
Z_w\\ 
1
\end{bmatrix}
\end{equation*}

\begin{equation*}
\tilde{p_i} = C \cdot \tilde{P_i}
\end{equation*}

Our goal in camera calibration is to discover this 12 parameters in the camera matrix. For this, we need a set of control points with 2D-3D correspondences. In other words, given a point in scene, your location in the image in pixels coordinates is known. 

---

## Tsai's method
This method consist of estimate the camera matrix using 3D control points of an object with know geometry and their 2D projections. Using an object with know geometry means that the 3D points in world coordinate frame are know. 

### Step 01 - Form the linear system: 

\begin{equation*}
Q = 
\begin{bmatrix}
X_w^1 &Y_w^1  &Z_w^1  &1  &0  &0  &0  &0  &-u_1 X_w^1 &-u_1 Y_w^1 &-u_1Z_w^1  &-u_1 \\ 
0 &0  &0  &0  &X_w^1 &Y_w^1  &Z_w^1  &1 &-v_1 X_w^1 &-v_1 Y_w^1 &-v_1Z_w^1  &-v_1\\ 
 &  &  &  &  &  &  &\vdots   &  &  &  & \\ 
X_w^n &Y_w^n  &Z_w^n  &1  &0  &0  &0  &0  &-u_n X_w^n &-u_n Y_w^n &-u_n Z_w^n  &-u_n \\ 
0 &0  &0  &0  &X_w^n &Y_w^n  &Z_w^n  &1 &-v_n X_w^n &-v_n Y_w^n &-v_nZ_w^n  &-v_n 
\end{bmatrix}

\end{equation*}

\begin{equation*}
Q \cdot c = 0
\end{equation*}

- $Q$ is a known coefficient matrix.
- $c$ is a column vector containing all the Camera matrix paramenters.

### Step 02 - Find $c$

This vector $c$ is found computing the SVD from $Q$: 

\begin{equation*}
    U\Sigma V^T = \text{SVD}(Q)
\end{equation*}

- $c$ correspond to the last column from $V$ matrix.

After found $c$, reconstruct the camera matrix $C$.

### Step 03 - Non-linear optimization

With the computed camera matrix $C$, you can project the world points and get new image points. From a otimal computed camera matrix $C$, the projected image point will be equal to the detected image points used to initially compute $C$. 

In realiaty, the projected image points will deviate from the detected image points. This difference is the reprojection error. Our goal is to estimate the camera parameters which minimize the reprojection error. 

For this, we use non-linear optimization techniques.

---

## Validate the implemented algorithm

For this, we create a virtual envienvironment for simulate a camera and the control points.


Create a camera with known parameters: 
- focal lenght $f$
- image properties 
- camera position
With this parameters, we can create the camera matrix for our virtual camera. 

Choose some world points and project then using `cv2.projectPoints()`.

For this set of control points, compute the camera matrix $C$ using the implement DLT algorithm. 

Compare the original camera matrix and the estimated $C$ by measuring the difference between the original image points and the projected image points by the estimated $C$. 

To visualize the process and the creation of the virtual enviroment, use some plot methods in `plot.py`

---


## Tentativa própria de implementação da simulação

In [3]:
#Import the librarys
import numpy as np

from dlt import calibrate_camera, create_image_points, create_world_points

## Tentativa de simulação usando Machine Vision Toolbox

In [ ]:
from machinevisiontoolbox import CentralCamera
# Create simulate data using Machine Vision Toolbox
cam = CentralCamera(f=0.015, rho=10e-6, imagesize=[1280, 1024], pp=[640, 512], name="mycamera")

cam.plot()

'''w_points = np.array([[1., 0.,1],[1,1,1]])
i_points = cam.project_point(w_points)
print(i_points)'''

